In [1]:
from src.dataset import Darknet
from src.models import ClassificationModel
from src.utils.functions import mkdir_if_not_exists, verify_existence_data
import tensorflow as tf
from tqdm import tqdm

In [2]:
data_path = f'dataset/Darknet/'
#TODO {colocar para pegar o dataset por parâmetro}
darknet_dataset = Darknet(data_path)
if not verify_existence_data(f"{data_path}/processed/darknet_dataset_processed_encoded.csv") and not verify_existence_data(f"{data_path}/processed/darknet_dataset_processed.csv"):
    darknet_dataset.exportProcessedData()
darknet_data, darknet_data_models = darknet_dataset.getProcessedData()

In [3]:
train, test, val = darknet_dataset.getTrainTestValData(0.33, True)

In [4]:
train_ds = darknet_dataset.convertToDataset(train, batch_size=256)
test_ds = darknet_dataset.convertToDataset(test, shuffle=False, batch_size=256)
val_ds = darknet_dataset.convertToDataset(val, shuffle=False, batch_size=256)

In [5]:
data_columns = darknet_data_models.columns
integer_category_columns = data_columns[:108]
real_columns = data_columns[108:len(data_columns) - 2]

inputs = []
encoded_features =[]

In [6]:
print("[!] - Normalizating real columns", end="\n")
for i in tqdm(range(len(real_columns))):
    numeric_col = tf.keras.Input(shape=(1, ), name=real_columns[i])
    normalization_layer = darknet_dataset.getNormalizationLayer(real_columns[i], train_ds)
    encoded_numeric_col = normalization_layer(numeric_col)
    inputs.append(numeric_col)
    encoded_features.append(encoded_numeric_col)

  0%|          | 0/76 [00:00<?, ?it/s]

[!] - Normalizating real columns


100%|██████████| 76/76 [14:02<00:00, 11.09s/it]


In [7]:
print("[!] - Encoding integer and string layers", end="\n")
for i in tqdm(range(len(integer_category_columns))):
    categorical_col = tf.keras.Input(shape=(1, ), name=integer_category_columns[i], dtype='int64')
    encoding_layer = darknet_dataset.getCategoryEncodingLayer(integer_category_columns[i], train_ds, dtype='int64', max_tokens=5)
    encoded_categorical_col = encoding_layer(categorical_col)
    inputs.append(categorical_col)
    encoded_features.append(encoded_categorical_col)

  0%|          | 0/108 [00:00<?, ?it/s]

[!] - Encoding integer and string layers


100%|██████████| 108/108 [28:10<00:00, 15.65s/it]


In [8]:
all_features = tf.keras.layers.concatenate(encoded_features)

In [12]:
x = tf.keras.layers.Dense(32, activation='relu')(all_features)
x = tf.keras.layers.Dropout(0.5)(x)
output = tf.keras.layers.Dense(1)(x)
model = tf.keras.Model(inputs, output)
model.compile(optimizer='adam', loss=tf.keras.losses.BinaryCrossentropy(from_logits=True), metrics=['accuracy'])


In [13]:
fit_model = model.fit(train_ds, epochs=20, validation_data = val_ds)
print(fit_model.history.keys())

Epoch 1/20


c:\users\pvbr0\anaconda3\envs\deep_learning_classification\lib\site-packages\tensorflow\python\keras\engine\functional.py:588: UserWarning: Input dict contained keys ['Label'] which did not match any model input. They will be ignored by the model.
  warnings.warn(


249/249 [==============================] - 49s 147ms/step - loss: 0.1176 - accuracy: 0.9635 - val_loss: 0.0191 - val_accuracy: 0.9957

In [15]:
loss, acc = model.evaluate(test_ds)
print(loss, acc)

183/183 [==============================] - 14s 76ms/step - loss: 0.0071 - accuracy: 0.9988

In [16]:
print(model.metrics_names)

['loss', 'accuracy']


In [17]:
model.save('etc/train_model/origin_classifier')

INFO:tensorflow:Assets written to: etc/train_model/origin_classifier\assets


INFO:tensorflow:Assets written to: etc/train_model/origin_classifier\assets


In [18]:
x_c = tf.keras.layers.Dense(128, activation='relu')(all_features)
x_c = tf.keras.layers.Dense(64, activation='relu')(x_c)
output = tf.keras.layers.Dense(8, activation='softmax')(x_c)
model_c = tf.keras.Model(inputs, output)
model_c.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

In [19]:
fit_model_c = model_c.fit(train_ds, epochs=20, validation_data = val_ds)
print(fit_model_c.history.keys())

Epoch 1/20


c:\users\pvbr0\anaconda3\envs\deep_learning_classification\lib\site-packages\tensorflow\python\keras\engine\functional.py:588: UserWarning: Input dict contained keys ['Label'] which did not match any model input. They will be ignored by the model.
  warnings.warn(


ValueError: in user code:

    c:\users\pvbr0\anaconda3\envs\deep_learning_classification\lib\site-packages\tensorflow\python\keras\engine\training.py:855 train_function  *
        return step_function(self, iterator)
    c:\users\pvbr0\anaconda3\envs\deep_learning_classification\lib\site-packages\tensorflow\python\keras\engine\training.py:845 step_function  **
        outputs = model.distribute_strategy.run(run_step, args=(data,))
    c:\users\pvbr0\anaconda3\envs\deep_learning_classification\lib\site-packages\tensorflow\python\distribute\distribute_lib.py:1285 run
        return self._extended.call_for_each_replica(fn, args=args, kwargs=kwargs)
    c:\users\pvbr0\anaconda3\envs\deep_learning_classification\lib\site-packages\tensorflow\python\distribute\distribute_lib.py:2833 call_for_each_replica
        return self._call_for_each_replica(fn, args, kwargs)
    c:\users\pvbr0\anaconda3\envs\deep_learning_classification\lib\site-packages\tensorflow\python\distribute\distribute_lib.py:3

In [ ]:
loss, acc = model_c.evaluate(test_ds)
print(loss, acc)

In [ ]:
model_c.save('etc/train_model/origin_characterization')

